In [1]:
from os.path import join, isfile, isdir
from os import listdir
import pandas as pd

In [2]:
import random
import numpy as np

from joblib import load
from utils import seed
from sklearn.model_selection import train_test_split
from teacher.datasets import load_iris, load_wine, load_beer

random_state = seed

In [3]:
DATASETS = {
    'iris': load_iris,
    'wine': load_wine,
    'beer': load_beer
}

In [4]:
BLACKBOX = ['RF', 'SVM', 'NN']

In [5]:
# %%
random.seed(random_state)
np.random.seed(random_state)

n_path_models = './models/'

blackbox_accuracy = {}

for ds in DATASETS:
    for black_box in BLACKBOX:
        bb = load(n_path_models + '%s_%s.joblib' % (ds, black_box))
        dataset = DATASETS[ds]()

        X, y = dataset['X'], dataset['y']

        # TRAIN 60, DEV 30, TEST 10
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, random_state=random_state)
        X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, train_size=0.75, random_state=random_state)

        blackbox_accuracy[(ds, black_box)] = bb.score(X_test, y_test)



In [6]:
blackbox_accuracy

{('iris', 'RF'): 0.9333333333333333,
 ('iris', 'SVM'): 1.0,
 ('iris', 'NN'): 1.0,
 ('wine', 'RF'): 1.0,
 ('wine', 'SVM'): 0.6666666666666666,
 ('wine', 'NN'): 0.8888888888888888,
 ('beer', 'RF'): 1.0,
 ('beer', 'SVM'): 0.85,
 ('beer', 'NN'): 0.8}

In [7]:
# FDT STATS
random.seed(random_state)
np.random.seed(random_state)

n_path_models = './models/'

fdt_stats = []

for ds in DATASETS:
    bb = load(n_path_models + '%s_%s.joblib' % (ds, 'FDT'))
    dataset = DATASETS[ds]()
    class_name = dataset['class_name']
    df = dataset['df']
    X = df.drop(class_name, axis=1)
    y = df[class_name]

    # TRAIN 60, DEV 30, TEST 10
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, random_state=random_state)
    X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, train_size=0.75, random_state=random_state)
    rs = bb.to_rule_based_system()
    rs_length = len(rs)
    rs_rule_length = np.mean([len(r.antecedent) for r in rs])

    row = {
        'Dataset': ds,
        'Method': 'FDT',
        'Blackbox': '-',
        'seed': 0,
        'Accuracy': bb.score(X_test, y_test),
        '#Rules': rs_length,
        '#Premises': rs_rule_length
    }
    fdt_stats.append(row)

In [8]:
fdt_stats

[{'Dataset': 'iris',
  'Method': 'FDT',
  'Blackbox': '-',
  'seed': 0,
  'Accuracy': 1.0,
  '#Rules': 12,
  '#Premises': 1.4166666666666667},
 {'Dataset': 'wine',
  'Method': 'FDT',
  'Blackbox': '-',
  'seed': 0,
  'Accuracy': 0.9444444444444444,
  '#Rules': 36,
  '#Premises': 3.0},
 {'Dataset': 'beer',
  'Method': 'FDT',
  'Blackbox': '-',
  'seed': 0,
  'Accuracy': 1.0,
  '#Rules': 69,
  '#Premises': 2.420289855072464}]

In [10]:
my_path = './results/'
onlyfiles = [f for f in listdir(my_path) if isfile(join(my_path, f)) and 'seed' in f]

In [11]:
res = []
for f in onlyfiles:
    with open(join(my_path, f), 'r') as file:
        try:
            params = f[:-4].split('_')
            seed = params[2]
            db = params[3]
            method = params[4]
            population_size = params[5]
            size_pressure = params[6]
            kappa = params[7]
            epsilon = params[8]
            bb = params[9]
            best_flocal_size, best_flocal_score, best_flocal_rule_size, best_flocal_fidelity, flocal_score, flocal_size, flocal_rule_size, flocal_fidelity = [float(i) for i in file.read()[1:-1].split(', ')]
            res.append([best_flocal_size, best_flocal_score, best_flocal_rule_size, best_flocal_fidelity, flocal_score, flocal_size, flocal_rule_size, flocal_fidelity, seed, db, method, bb])
        except:
            pass


In [12]:
df = pd.DataFrame(res, columns=['#Rules-FLX', 'Accuracy-FLX', '#Premises-FLX', 'Fidelity-FLX', 'Accuracy', '#Rules', '#Premises', 'Fidelity', 'seed', 'Dataset', 'Method', 'Blackbox'])

In [13]:
df = pd.concat([df, pd.DataFrame(fdt_stats)], ignore_index=True)

In [14]:
for ds, bb in blackbox_accuracy:
    df.loc[(df['Dataset'] == ds) & (df['Blackbox'] == bb), 'Accuracy-BB'] = blackbox_accuracy[(ds, bb)]

In [15]:
df['Method'] = df['Method'].str.upper()
df['Dataset'] = df['Dataset'].str.title()

In [16]:
df['seed'] = pd.to_numeric(df['seed'])

In [17]:
df

,#Rules-FLX,Accuracy-FLX,#Premises-FLX,Fidelity-FLX,Accuracy,#Rules,#Premises,Fidelity,seed,Dataset,Method,Blackbox,Accuracy-BB
0,14.0,0.825000,2.000000,0.650000,0.725000,119.0,2.008403,0.750000,10,Beer,LORE,NN,0.800000
1,19.0,0.900000,2.368421,0.900000,0.950000,119.0,2.537815,0.950000,10,Beer,LORE,RF,1.000000
2,16.0,0.750000,2.000000,0.650000,0.700000,119.0,2.016807,0.775000,10,Beer,LORE,SVM,0.850000
3,4.0,1.000000,1.750000,1.000000,1.000000,45.0,1.955556,1.000000,10,Iris,LORE,NN,1.000000
4,4.0,0.933333,2.000000,0.933333,1.000000,45.0,2.066667,1.000000,10,Iris,LORE,RF,0.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,2.0,0.666667,1.000000,0.666667,0.666667,41.0,1.390244,0.666667,9,Wine,FLARE,RF,1.000000
341,5.0,0.777778,1.200000,0.555556,0.722222,17.0,1.000000,0.944444,9,Wine,FLARE,SVM,0.666667
342,NaN,NaN,NaN,NaN,1.000000,12.0,1.416667,NaN,0,Iris,FDT,-,NaN
343,NaN,NaN,NaN,NaN,0.944444,36.0,3.000000,NaN,0,Wine,FDT,-,NaN


In [18]:
df.groupby(['Dataset', 'Method', 'Blackbox']).mean()[['#Rules', '#Rules-FLX', '#Premises', '#Premises-FLX']].round(2)

#Rules  #Rules-FLX  #Premises  #Premises-FLX
Dataset Method Blackbox                                              
Beer    FDT    -           69.0         NaN       2.42            NaN
        FLARE  NN         128.0       20.37       1.85           1.79
               RF         129.0       27.63       2.34           2.22
               SVM         99.0       15.89       1.96           1.91
        LORE   NN         119.0       14.58       2.01           1.99
               RF         119.0       15.84       2.54           2.53
               SVM        119.0       13.68       2.02           2.04
Iris    FDT    -           12.0         NaN       1.42            NaN
        FLARE  NN          32.0        5.05       1.31           1.27
               RF          26.0        4.16       1.81           1.70
               SVM         19.0        4.47       1.26           1.32
        LORE   NN          45.0        4.79       1.96           1.55
               RF          45.0        4.16       2.07           1.85
               SVM         45.0        4.37       1.58           1.36
Wine    FDT    -           36.0         NaN       3.00            NaN
        FLARE  NN          48.0        8.68       1.42           1.54
               RF          41.0        2.89       1.39           1.23
               SVM         17.0        4.95       1.00           1.32
        LORE   NN          54.0        4.79       2.52           1.94
               RF          54.0        6.21       3.19           3.02
               SVM         52.0        5.05       1.02           1.56

In [18]:
df.groupby(['Dataset', 'Method', 'Blackbox']).mean()[['Fidelity', 'Fidelity-FLX', 'Accuracy-BB', 'Accuracy', 'Accuracy-FLX']].round(2)

Fidelity  Fidelity-FLX  Accuracy-BB  Accuracy  \
Dataset Method Blackbox                                                  
Beer    FDT    -              NaN           NaN          NaN      0.42   
        FLARE  NN            0.69          0.70         0.80      0.67   
               RF            0.87          0.88         1.00      0.87   
               SVM           0.86          0.76         0.85      0.85   
        LORE   NN            0.74          0.78         0.80      0.70   
               RF            0.92          0.88         1.00      0.92   
               SVM           0.78          0.74         0.85      0.68   
Iris    FDT    -              NaN           NaN          NaN      1.00   
        FLARE  NN            0.97          0.93         1.00      0.95   
               RF            0.94          0.91         0.93      0.94   
               SVM           0.93          0.95         1.00      0.95   
        LORE   NN            0.93          0.94         1.00      0.94   
               RF            0.97          0.93         0.93      0.97   
               SVM           0.92          0.95         1.00      0.94   
Wine    FDT    -              NaN           NaN          NaN      0.94   
        FLARE  NN            0.86          0.76         0.89      0.82   
               RF            0.61          0.61         1.00      0.61   
               SVM           0.99          0.73         0.67      0.71   
        LORE   NN            0.77          0.78         0.89      0.76   
               RF            0.90          0.88         1.00      0.90   
               SVM           0.93          0.76         0.67      0.68   

                         Accuracy-FLX  
Dataset Method Blackbox                
Beer    FDT    -                  NaN  
        FLARE  NN                0.74  
               RF                0.88  
               SVM               0.82  
        LORE   NN                0.82  
               RF                0.87  
               SVM               0.79  
Iris    FDT    -                  NaN  
        FLARE  NN                0.91  
               RF                0.91  
               SVM               0.92  
        LORE   NN                0.93  
               RF                0.93  
               SVM               0.92  
Wine    FDT    -                  NaN  
        FLARE  NN                0.77  
               RF                0.61  
               SVM               0.76  
        LORE   NN                0.76  
               RF                0.88  
               SVM               0.71

In [19]:
print(df.groupby(['Dataset', 'Method', 'Blackbox']).mean()[['#Rules', '#Rules-FLX', '#Premises', '#Premises-FLX']].to_latex(float_format="%.2f", escape=True, multirow=True, column_format='lll|rrrr', na_rep='-'))

\begin{tabular}{lll|rrrr}
\toprule
 &  &  & \#Rules & \#Rules-FLX & \#Premises & \#Premises-FLX \\
Dataset & Method & Blackbox &  &  &  &  \\
\midrule
\multirow[t]{7}{*}{Beer} & FDT & - & 69.00 & - & 2.42 & - \\
\cline{2-7}
 & \multirow[t]{3}{*}{FLARE} & NN & 128.00 & 20.37 & 1.85 & 1.79 \\
 &  & RF & 129.00 & 27.63 & 2.34 & 2.22 \\
 &  & SVM & 99.00 & 15.89 & 1.96 & 1.91 \\
\cline{2-7}
 & \multirow[t]{3}{*}{LORE} & NN & 119.00 & 14.58 & 2.01 & 1.99 \\
 &  & RF & 119.00 & 15.84 & 2.54 & 2.53 \\
 &  & SVM & 119.00 & 13.68 & 2.02 & 2.04 \\
\cline{1-7} \cline{2-7}
\multirow[t]{7}{*}{Iris} & FDT & - & 12.00 & - & 1.42 & - \\
\cline{2-7}
 & \multirow[t]{3}{*}{FLARE} & NN & 32.00 & 5.05 & 1.31 & 1.27 \\
 &  & RF & 26.00 & 4.16 & 1.81 & 1.70 \\
 &  & SVM & 19.00 & 4.47 & 1.26 & 1.32 \\
\cline{2-7}
 & \multirow[t]{3}{*}{LORE} & NN & 45.00 & 4.79 & 1.96 & 1.55 \\
 &  & RF & 45.00 & 4.16 & 2.07 & 1.85 \\
 &  & SVM & 45.00 & 4.37 & 1.58 & 1.36 \\
\cline{1-7} \cline{2-7}
\multirow[t]{7}{*}{Wine} &

In [20]:
print(df.groupby(['Dataset', 'Method', 'Blackbox']).mean()[['Fidelity', 'Fidelity-FLX', 'Accuracy-BB', 'Accuracy', 'Accuracy-FLX']].to_latex(float_format="%.2f", escape=True, multirow=True, column_format='lll|rrrrr', na_rep='-'))

\begin{tabular}{lll|rrrrr}
\toprule
 &  &  & Fidelity & Fidelity-FLX & Accuracy-BB & Accuracy & Accuracy-FLX \\
Dataset & Method & Blackbox &  &  &  &  &  \\
\midrule
\multirow[t]{7}{*}{Beer} & FDT & - & - & - & - & 0.42 & - \\
\cline{2-8}
 & \multirow[t]{3}{*}{FLARE} & NN & 0.69 & 0.70 & 0.80 & 0.67 & 0.74 \\
 &  & RF & 0.87 & 0.88 & 1.00 & 0.87 & 0.88 \\
 &  & SVM & 0.86 & 0.76 & 0.85 & 0.85 & 0.82 \\
\cline{2-8}
 & \multirow[t]{3}{*}{LORE} & NN & 0.74 & 0.78 & 0.80 & 0.70 & 0.82 \\
 &  & RF & 0.92 & 0.88 & 1.00 & 0.92 & 0.87 \\
 &  & SVM & 0.78 & 0.74 & 0.85 & 0.67 & 0.79 \\
\cline{1-8} \cline{2-8}
\multirow[t]{7}{*}{Iris} & FDT & - & - & - & - & 1.00 & - \\
\cline{2-8}
 & \multirow[t]{3}{*}{FLARE} & NN & 0.97 & 0.93 & 1.00 & 0.95 & 0.91 \\
 &  & RF & 0.94 & 0.91 & 0.93 & 0.94 & 0.91 \\
 &  & SVM & 0.93 & 0.95 & 1.00 & 0.95 & 0.92 \\
\cline{2-8}
 & \multirow[t]{3}{*}{LORE} & NN & 0.93 & 0.94 & 1.00 & 0.94 & 0.93 \\
 &  & RF & 0.97 & 0.93 & 0.93 & 0.97 & 0.93 \\
 &  & SVM & 0.92 & 0.

In [20]:
print(df.groupby(['Dataset', 'Method', 'Blackbox']).mean()[['Fidelity', 'Fidelity-FLX', 'Accuracy-BB', 'Accuracy', 'Accuracy-FLX', '#Rules', '#Rules-FLX', '#Premises', '#Premises-FLX']].to_latex(float_format="%.2f", escape=True, multirow=True, column_format='lll|rrrrr|rrrr', na_rep='-'))

\begin{tabular}{lll|rrrrr|rrrr}
\toprule
 &  &  & Fidelity & Fidelity-FLX & Accuracy-BB & Accuracy & Accuracy-FLX & \#Rules & \#Rules-FLX & \#Premises & \#Premises-FLX \\
Dataset & Method & Blackbox &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{7}{*}{Beer} & FDT & - & - & - & - & 1.00 & - & 69.00 & - & 2.42 & - \\
\cline{2-12}
 & \multirow[t]{3}{*}{FLARE} & NN & 0.69 & 0.70 & 0.80 & 0.67 & 0.74 & 128.00 & 20.37 & 1.85 & 1.79 \\
 &  & RF & 0.87 & 0.88 & 1.00 & 0.87 & 0.88 & 129.00 & 27.63 & 2.34 & 2.22 \\
 &  & SVM & 0.86 & 0.76 & 0.85 & 0.85 & 0.82 & 99.00 & 15.89 & 1.96 & 1.91 \\
\cline{2-12}
 & \multirow[t]{3}{*}{LORE} & NN & 0.74 & 0.78 & 0.80 & 0.70 & 0.82 & 119.00 & 14.58 & 2.01 & 1.99 \\
 &  & RF & 0.92 & 0.88 & 1.00 & 0.92 & 0.87 & 119.00 & 15.84 & 2.54 & 2.53 \\
 &  & SVM & 0.78 & 0.74 & 0.85 & 0.67 & 0.79 & 119.00 & 13.68 & 2.02 & 2.04 \\
\cline{1-12} \cline{2-12}
\multirow[t]{7}{*}{Iris} & FDT & - & - & - & - & 1.00 & - & 12.00 & - & 1.42 & - \\
\cline{2-12}
 & \multiro